# F20 Intro to AI: Assignment 1, Part I 

**Audrey Zhang**

**September 12, 2020**

In [6]:
# set up 

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import os
import numpy as np
import pandas as pd

Define get_location and get_distance_matrix functions to use assignment datasets for this problem

In [7]:
def get_locations(loc_filepath = 'C:/Users/audre/OneDrive/Documents/CMU/Fall_2020/Intro_to_AI/assignment_1/locations.txt'):
    names=[]
    with open(loc_filepath, 'rt', encoding='utf-8') as fin:
        for line in fin:
            names.append(line.rstrip())
    return names

In [8]:
def get_distance_matrix(dist_filepath = 'C:/Users/audre/OneDrive/Documents/CMU/Fall_2020/Intro_to_AI/assignment_1/distances.csv'):
    '''
    Read the file and return a list of lists or a 2D array with 
    distances such that dist_matrix[loc1][loc2] returns the distance from loc1 to loc2
    e.g. dist_matrix = [[0,4,9],
                        [4,0,8],
                        [9,8,0]] for 3  sample locations
    '''
    distances=pd.read_csv(dist_filepath, sep=',', header=None)
    distances_arr=np.array(distances)
    return distances_arr

In [9]:
# create and save list of locations to print final route 
locations_list=get_locations() 

Run code from Google TSP algorithm

In [10]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Edited here to use provided distance matrix
    data['distance_matrix'] = get_distance_matrix()
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

Note: edited slightly below to output names of locations on route, along with location indices, in print_solution().

In [11]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_locations=''
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        """AZ EDIT: add location names"""
        route_locations+=' {} ->'.format(locations_list[manager.IndexToNode(index)]) 
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    route_locations+=' {}\n'.format(locations_list[manager.IndexToNode(index)])   
    plan_output += ' {}'.format(manager.IndexToNode(index))
    print(plan_output)
    print(route_locations)
    plan_output += 'Route distance: {} miles\n'.format(route_distance)

In [12]:
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)

In [13]:
if __name__ == '__main__':
    main()

Objective: 42980 miles
Route for vehicle 0:
 0 -> 3 -> 8 -> 1 -> 4 -> 7 -> 5 -> 6 -> 2 -> 9 -> 0
 St. Stephan's Cathedral, Austria -> Brugge, Belgium -> Edinburgh, Scotland -> Teide, Spain -> Montreal, Canada -> Brisbane, Australia -> Itsukushima Shrine, Japan -> Shanghai, China -> Tallinn, Estonia -> Stockholm, Sweden -> St. Stephan's Cathedral, Austria

